In [1]:
import pandas as pd
import pickle as pkl
import tensorflow as tf
import time

In [2]:
def model_fn(features, labels, mode):
    emb_size = 50
    filters=10
    kernel_size=3
    
    with open("data/dictionary.pkl", "rb") as f:
        dictionary = pkl.load(f)
    
    values = list(range(len(dictionary)))
    keys = list(dictionary)
    
    table = tf.contrib.lookup.HashTable(
      tf.contrib.lookup.KeyValueTensorInitializer(keys, values), -1
    )
    
    word_embeddings = tf.get_variable(
        "word_embeddings",
        shape=[len(dictionary), emb_size]
    )
    
    u_inputs = features["user_input"]
    i_inputs = features["item_input"]
    
    u_inputs = table.lookup(u_inputs)
    i_inputs = table.lookup(i_inputs)

    u_inputs = tf.nn.embedding_lookup(word_embeddings, u_inputs)
    i_inputs = tf.nn.embedding_lookup(word_embeddings, i_inputs)
    
    user_conv1 = tf.layers.conv1d(
        u_inputs,
        filters,
        kernel_size,
        use_bias=True,
        activation=tf.nn.tanh,
        name="user_conv")

    item_conv1 = tf.layers.conv1d(
        i_inputs,
        filters,
        kernel_size,
        use_bias=True,
        activation=tf.nn.tanh,
        name="item_conv")

    user_max_pool1 = tf.layers.max_pooling1d(user_conv1, 2, 1)
    item_max_pool1 = tf.layers.max_pooling1d(item_conv1, 2, 1)

    user_flat = tf.layers.flatten(user_max_pool1)
    item_flat = tf.layers.flatten(item_max_pool1)

    user_dense = tf.layers.dense(user_flat, 64, activation=tf.nn.relu)
    item_dense = tf.layers.dense(item_flat, 64, activation=tf.nn.relu)
    
    predictions = tf.reduce_sum( tf.multiply( user_dense, item_dense ), 1, keepdims=True )
    
    output = {
        "rating": predictions,
        "user_review_embedding": user_flat,
        "item_review_embedding": item_flat
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)


    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.mean_squared_error(labels, predictions)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=0.0001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
        "mean square error": tf.metrics.mean_squared_error(
            labels=labels, predictions=predictions)
    }
    return tf.estimator.EstimatorSpec(
        mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

# Training Functions

In [3]:
truncate_len = 800
batch_size = 32

In [4]:
def get_truncate_fn(trunc_len):
    def truncate_fn(user, item, rating):
        return user[:trunc_len], item[:trunc_len], rating
    return truncate_fn

In [5]:
def split_fn(user, item, rating):
    user = tf.string_split(user)
    item = tf.string_split(item)
    return user.values, item.values, rating

In [6]:
def parse_fn(record):
    features = {
            "user_review": tf.FixedLenSequenceFeature([], tf.string, allow_missing=True),
            "item_review": tf.FixedLenSequenceFeature([], tf.string, allow_missing=True),
            "rating": tf.FixedLenFeature([1], tf.float32)
        }
    parsed_features = tf.parse_single_example(record, features)
    return parsed_features["user_review"], parsed_features["item_review"], parsed_features["rating"]

In [7]:
def get_dataset_iterator(loc, batch_size, max_len, pad_value):
    dataset = tf.data.TFRecordDataset(loc)
    dataset = dataset.prefetch(batch_size)
    dataset = dataset.map(parse_fn, num_parallel_calls=batch_size)
    dataset = dataset.map(split_fn, num_parallel_calls=batch_size)
    dataset = dataset.map(get_truncate_fn(max_len), num_parallel_calls=batch_size)
    dataset = dataset.padded_batch(batch_size, padded_shapes=([max_len], [max_len], [None]), padding_values=(pad_value, pad_value, 0.0))
    dataset = dataset.shuffle(26352, reshuffle_each_iteration=False)
    iterator = dataset.make_one_shot_iterator()
    return iterator

In [8]:
def train_input_fn():
    train_dataset = get_dataset_iterator(
        loc="data/train.tfrecords",
        batch_size=batch_size,
        max_len=truncate_len,
        pad_value="unk")
    nex = train_dataset.get_next()
    return {"user_input": nex[0], "item_input": nex[1]}, tf.cast(nex[2], tf.int32)

In [9]:
def test_input_fn():
    test_dataset = get_dataset_iterator(
        loc="data/test.tfrecords",
        batch_size=batch_size,
        max_len=truncate_len,
        pad_value="unk"
    )
    nex = test_dataset.get_next()
    return (nex[0], nex[1]), nex[2]

In [10]:
model_dir = "dlprof/" + str(int(time.time()))
scoring_function = tf.estimator.Estimator(
    model_fn=model_fn,
    model_dir=model_dir)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'dlprof/1525550398', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f98822924a8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [11]:
s = time.time()
scoring_function.train(input_fn=train_input_fn)
e = time.time()
print("ran {} seconds".format(e - s))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into dlprof/1525550398/model.ckpt.
INFO:tensorflow:loss = 13.210334, step = 0
INFO:tensorflow:global_step/sec: 71.8545
INFO:tensorflow:loss = 1.9414926, step = 100 (1.393 sec)
INFO:tensorflow:global_step/sec: 75.6699
INFO:tensorflow:loss = 1.089664, step = 200 (1.321 sec)
INFO:tensorflow:global_step/sec: 71.5313
INFO:tensorflow:loss = 0.7747723, step = 300 (1.398 sec)
INFO:tensorflow:global_step/sec: 74.0304
INFO:tensorflow:loss = 1.2600307, step = 400 (1.351 sec)
INFO:tensorflow:global_step/sec: 70.2353
INFO:tensorflow:loss = 2.2621274, step = 500 (1.425 sec)
INFO:tensorflow:global_step/sec: 71.1435
INFO:tensorflow:loss = 1.5349941, step = 600 (1.404 sec)
INFO:tensorflow:global_step/sec: 74.3001
INFO:tensorflow:

In [12]:
def  serving_input_fn():
    input_placeholder = tf.placeholder(tf.string, shape=None)
    receiver_tensors = {
        "user_input": tf.placeholder(tf.string, shape=None),
        "item_input": tf.placeholder(tf.string, shape=None)
    }
    features = tf.parse_single_example(
        input_placeholder,
        {
            "user_input": tf.FixedLenSequenceFeature([truncate_len], tf.string, allow_missing=True),
            "item_input": tf.FixedLenSequenceFeature([truncate_len], tf.string, allow_missing=True)
        }
    )
    return tf.estimator.export.ServingInputReceiver(features, receiver_tensors)

In [13]:
scoring_function.export_savedmodel(
    model_dir,
    serving_input_fn
)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.


ValueError: export_outputs must be a dict and not<class 'NoneType'>